In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt

import wrangle as w
import explore as e
import env

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer, PolynomialFeatures

import warnings
warnings.filterwarnings("ignore")

In [2]:
#a = w.get_new_zillow_data()

In [3]:
#a.info()

In [4]:
df = w.wrangle_zillow()

In [5]:
df.head()

,bedrooms,bathrooms,sq_feet,tax_value,year_built,tax_amount,fips
0,4,3.5,3100,1023282,1998,11013.72,6059
1,2,1.0,1465,464000,1967,5672.48,6111
2,3,2.0,1243,564778,1962,6488.30,6059
3,4,3.0,2376,145143,1970,1777.51,6037
4,4,3.0,2962,773303,1950,9516.26,6037


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50790 entries, 0 to 50789
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bedrooms    50790 non-null  int64  
 1   bathrooms   50790 non-null  float64
 2   sq_feet     50790 non-null  int64  
 3   tax_value   50790 non-null  int64  
 4   year_built  50790 non-null  int64  
 5   tax_amount  50790 non-null  float64
 6   fips        50790 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 2.7 MB


In [7]:
train, validate, test = w.split_data(df)

In [8]:
train_scaled, validate_scaled, test_scaled = w.scale_data(train, validate, test)

In [9]:
train_scaled.head(3)

,bedrooms,bathrooms,sq_feet,tax_value,year_built,tax_amount,fips
8096,0.500000,0.333333,0.162985,206483,1983,0.036235,6111
45007,0.333333,0.166667,0.152955,110048,1970,0.023914,6037
43774,0.500000,0.333333,0.171701,82423,1956,0.018175,6059


In [10]:
train_scaled.dtypes

bedrooms      float64
bathrooms     float64
sq_feet       float64
tax_value       int64
year_built      int64
tax_amount    float64
fips            int64
dtype: object

In [11]:
#x_cols = train_scaled[['bedrooms','bathrooms','sq_feet']]

In [12]:
#y_cols = train_scaled.tax_value

In [13]:
#x_cols.dtypes

In [14]:
#y_cols.dtypes

In [15]:
#x_cols_list = list(x_cols)

In [16]:
#x_cols[x_cols_list[0]]

In [17]:
#for i in x_cols[1:]:
#    print(i)
 #   plt.figure()
  #  plt.plot(y_cols,x_cols[i])
  #  plt.show()

In [18]:
#e.plot_categorical_and_continuous_vars(train_scaled, x_cols,y_cols)

In [19]:
#e.plot_variable_pairs(x_cols, y_cols)

In [20]:
#x_cols['bedrooms']

In [21]:
x_train_scaled = train_scaled[['bedrooms', 'bathrooms', 'sq_feet']]
y_train = train[['tax_value']]

x_validate_scaled = validate_scaled[['bedrooms', 'bathrooms', 'sq_feet']]
y_validate = validate[['tax_value']]

x_test_scaled = test_scaled[['bedrooms', 'bathrooms', 'sq_feet']]
y_test = test[['tax_value']]

In [22]:
y_train.head()

,tax_value
8096,206483
45007,110048
43774,82423
1082,373151
13597,1430885


# Modeling AND Evaluation

# Baseline

In [23]:
def get_baseline(x_train_scaled, y_train):
    y_train['tax_value_pred_mean']= y_train['tax_value'].mean()
    y_validate['tax_value_pred_mean']= y_validate['tax_value'].mean()

    y_train['tax_value_pred_median'] = y_train['tax_value'].median()
    y_validate['tax_value_pred_median'] = y_validate['tax_value'].median()

    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_mean)**(1/2)
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_mean)**(1/2)

    print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
          "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))
    
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_median)**(1/2)
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_median)**(1/2)

    print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
          "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

In [24]:
get_baseline(x_train_scaled, y_train)

RMSE using Mean
Train/In-Sample:  355371.19 
Validate/Out-of-Sample:  347152.8
RMSE using Median
Train/In-Sample:  364925.59 
Validate/Out-of-Sample:  355040.41


In [25]:
y_train.head()

,tax_value,tax_value_pred_mean,tax_value_pred_median
8096,206483,445957.819914,363000.0
45007,110048,445957.819914,363000.0
43774,82423,445957.819914,363000.0
1082,373151,445957.819914,363000.0
13597,1430885,445957.819914,363000.0


# LinearRegression (OLS)

In [26]:
def get_ols(x_train_scaled, y_train):
    # create the model object
    lm = LinearRegression(normalize=True)

    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    lm.fit(x_train_scaled, y_train.tax_value)

    # predict train
    y_train['tax_value_pred_lm'] = lm.predict(x_train_scaled)

    # evaluate: rmse
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_lm)**(1/2)

    # predict validate
    y_validate['tax_value_pred_lm'] = lm.predict(x_validate_scaled)

    # evaluate: rmse
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_lm)**(1/2)

    print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
          "\nValidation/Out-of-Sample: ", rmse_validate)

In [27]:
get_ols(x_train_scaled, y_train)

RMSE for OLS using LinearRegression
Training/In-Sample:  286205.31265758 
Validation/Out-of-Sample:  281570.92470836977


In [28]:
y_train.head()

,tax_value,tax_value_pred_mean,tax_value_pred_median,tax_value_pred_lm
8096,206483,445957.819914,363000.0,3.927581e+05
45007,110048,445957.819914,363000.0,3.825737e+05
43774,82423,445957.819914,363000.0,4.103669e+05
1082,373151,445957.819914,363000.0,2.477532e+05
13597,1430885,445957.819914,363000.0,1.416496e+06


# Lars

In [29]:
def get_lars(x_train_scaled, y_train):
    lars = LassoLars(alpha=1.0)
    #scaler = MinMaxScaler()
    #x_train_scaled = x_train_scaled.copy()
    #x_train_scaled[['bedrooms','bathrooms','sq_feet','year_built','tax_amount','fips']] = scaler.fit_transform(x_train_scaled)
    #x_validate_scaled = x_validate_scaled.copy()
    #x_validate_scaled[['bedrooms','bathrooms','sq_feet','year_built','tax_amount','fips']] = scaler.fit_transform(x_validate_scaled)

    lars.fit(x_train_scaled, y_train.tax_value)

    y_train['tax_value_pred_lars'] = lars.predict(x_train_scaled)

    # evaluate: rmse
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_lars)**(1/2)

    # predict validate
    y_validate['tax_value_pred_lars'] = lars.predict(x_validate_scaled)

    # evaluate: rmse
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_lars)**(1/2)

    print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
          "\nValidation/Out-of-Sample: ", rmse_validate)

In [30]:
get_lars(x_train_scaled, y_train)

RMSE for Lasso + Lars
Training/In-Sample:  286205.6184335364 
Validation/Out-of-Sample:  281570.55340928794


# TweedieRegressor (GLM)

In [31]:
def get_glm(x_train_scaled, y_train):
    
    # create the model object
    glm = TweedieRegressor(power=1, alpha=0)

    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    glm.fit(x_train_scaled, y_train.tax_value)

    # predict train
    y_train['tax_value_pred_glm'] = glm.predict(x_train_scaled)

    # evaluate: rmse
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_glm)**(1/2)

    # predict validate
    y_validate['tax_value_pred_glm'] = glm.predict(x_validate_scaled)

    # evaluate: rmse
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_glm)**(1/2)

    print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train, 
          "\nValidation/Out-of-Sample: ", rmse_validate)

In [32]:
get_glm(x_train_scaled, y_train)

RMSE for GLM using Tweedie, power=1 & alpha=0
Training/In-Sample:  296696.93800469773 
Validation/Out-of-Sample:  288237.9091112238


# Polynomial Regression

In [33]:
def get_poly(x_train_scaled, y_train):
    # make the polynomial features to get a new set of features
    pf = PolynomialFeatures(degree=2)

    # fit and transform X_train_scaled
    x_train_scaled_degree2 = pf.fit_transform(x_train_scaled)

    # transform X_validate_scaled & X_test_scaled
    x_validate_scaled_degree2 = pf.transform(x_validate_scaled)
    x_test_scaled_degree2 = pf.transform(x_test_scaled)
    
    # create the model object
    lm2 = LinearRegression(normalize=True)

    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    lm2.fit(x_train_scaled_degree2, y_train.tax_value)

    # predict train
    y_train['tax_value_pred_lm2'] = lm2.predict(x_train_scaled_degree2)

    # evaluate: rmse
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_lm2)**(1/2)

    # predict validate
    y_validate['tax_value_pred_lm2'] = lm2.predict(x_validate_scaled_degree2)

    # evaluate: rmse
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_lm2)**(1/2)

    print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
          "\nValidation/Out-of-Sample: ", rmse_validate)
    

In [34]:
get_poly(x_train_scaled, y_train)

RMSE for Polynomial Model, degrees=2
Training/In-Sample:  285135.8897185141 
Validation/Out-of-Sample:  280525.2753433108


In [35]:
y_train.head(3)

,tax_value,tax_value_pred_mean,tax_value_pred_median,tax_value_pred_lm,tax_value_pred_lars,tax_value_pred_glm,tax_value_pred_lm2
8096,206483,445957.819914,363000.0,392758.090123,392753.032081,377474.583001,387092.079603
45007,110048,445957.819914,363000.0,382573.677449,382330.711419,347714.519021,390425.469488
43774,82423,445957.819914,363000.0,410366.938957,410338.708267,387057.648601,404948.566514


# R^2 (check model predicted performance)

In [36]:
from sklearn.metrics import r2_score
def calc_performance(y, yhat, featureN = 2):
        # A version of R Squared which may be more accurate than when done by hand
        r2 = r2_score(y, yhat)
        # Adjusted R Squared
        adjR2= 1-(1-r2)*(len(y)-1)/(len(y)-featureN-1)
        return r2, adjR2

In [37]:
calc_performance(y_train.tax_value, y_train.tax_value_pred_lm2, featureN = 2)

(0.35621725972306284, 0.35617198508328807)

In [38]:
y_train.head()

,tax_value,tax_value_pred_mean,tax_value_pred_median,tax_value_pred_lm,tax_value_pred_lars,tax_value_pred_glm,tax_value_pred_lm2
8096,206483,445957.819914,363000.0,3.927581e+05,3.927530e+05,3.774746e+05,3.870921e+05
45007,110048,445957.819914,363000.0,3.825737e+05,3.823307e+05,3.477145e+05,3.904255e+05
43774,82423,445957.819914,363000.0,4.103669e+05,4.103387e+05,3.870576e+05,4.049486e+05
1082,373151,445957.819914,363000.0,2.477532e+05,2.480505e+05,2.970859e+05,2.565237e+05
13597,1430885,445957.819914,363000.0,1.416496e+06,1.414726e+06,2.115100e+06,1.651991e+06


In [39]:
calc_performance(y_test.tax_value, y_test.tax_value_pred_lm2)

AttributeError: 'DataFrame' object has no attribute 'tax_value_pred_lm2'

# Best Model On Test

> Best model is Polynomial Model, it moves to test

In [ ]:
def get_poly_test(x_train_scaled, y_test):
    # make the polynomial features to get a new set of features
    pf = PolynomialFeatures(degree=2)

    # fit and transform X_train_scaled
    x_train_scaled_degree2 = pf.fit_transform(x_train_scaled)

    # transform X_validate_scaled & X_test_scaled
    x_validate_scaled_degree2 = pf.transform(x_validate_scaled)
    x_test_scaled_degree2 = pf.transform(x_test_scaled)
    
    # create the model object
    lm2 = LinearRegression(normalize=True)

    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    lm2.fit(x_train_scaled_degree2, y_train.tax_value)

    # predict train
    y_train['tax_value_pred_lm2'] = lm2.predict(x_train_scaled_degree2)

    # evaluate: rmse
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_lm2)**(1/2)

    # predict test
    y_test['tax_value_pred_lm2'] = lm2.predict(x_test_scaled_degree2)

    # evaluate: rmse
    rmse_test = mean_squared_error(y_test.tax_value, y_test.tax_value_pred_lm2)**(1/2)

    print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
          "\nTest/Out-of-Sample: ", rmse_test)
    

In [ ]:
get_poly_test(x_train_scaled, y_test)

In [ ]:
calc_performance(y_test.tax_value, y_test.tax_value_pred_lm2, featureN = 2)